# Machete con Diferentes Test

In [1]:
# Autor: Andres Montes de Oca

# Instalacion de Paquetes
# !pip install pingouin
# !pip install scipy
# !pip install statsmodels


# Cargamos Librerias y Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy import stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tools.tools as smt


# Asthetics
sns.set(style='ticks', context='notebook', palette='colorblind', font_scale=1, color_codes=True)      

## RPY2
#### Python <====> R

In [18]:
# !pip install rpy2
import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri

from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector

In [13]:
# Importing packages R packages
base = importr('base')
utils = importr('utils')

# Select a mirror for R packages
utils.chooseCRANmirror(ind=1) 

<rpy2.rinterface_lib.sexp.NULLType object at 0x0000017FD771F580> [RTYPES.NILSXP]

In [14]:
# Installing packages R packages
packnames = ('ggplot2', 'hexbin')

# Selectively install what needs to be install.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [22]:
# Getting R Objects
pi = ro.r['pi']

dataCars.head()

,speed,dist
0,4,2
1,4,10
2,7,4
3,7,22
4,8,16


In [30]:
# Dataframe from Python to R
with (ro.default_converter + pandas2ri.converter).context():
  dataCarsR = ro.conversion.get_conversion().py2rpy(dataCars)

type(dataCarsR)
dataCarsR

speed,dist
...,...


## Cargas de Datasets

In [19]:
# Iris para probar las Key Assumptions
dataI = sns.load_dataset('iris')

# Grasa de Cerdos
dataC = pd.read_excel('Data/grasacerdos.xlsx', index_col='Obs') # Cargamos el Dataset
dataC = dataC.replace(to_replace=',', value='.', regex=True) # Reemplazo , por .
dataC = dataC.astype('float') # Transformo en float

# Damascos
# dataD = pd.read_csv('Data/Damascos.csv', index_col='VARIEDAD') # Incompleto

# Dataset para Regresion Lineal
dataP = pd.read_excel('Data/peso_edad_colest.xlsx')

# Modelo Lineal para ejemplos (se en detalle ma abajo)
dataP = sm.add_constant(dataP)
model_colest = smf.ols('colest ~ edad', data=dataP).fit()

# Modelo para Box-Cox usando dataset de cars
dataCars = pd.read_csv('Data/cars.csv')
model_cars = smf.ols('dist ~ speed', data=dataCars).fit()

#### Logbook

In [ ]:
# 31/05/23 -> Creacion de la Notebook
# 31/05/23 -> 1, 2, 3, 4, 5
# 06/06/23 -> 6 - Correlacion de Residuos
# 17/06/23 -> 7 - Transfomraciones Box Clox

## Machete

### 1) Linear Relationship
Relationship between independent var (x) and depedent var (y) is linear

In [ ]:
sns.scatterplot(data=dataI, x='petal_length', y='petal_width')
plt.show()

# Transformmations for making relationships linear. Polynomial or Exponential for any of both

### 2) Multicollonearity
Eeach independent variable should be independent from other independent variables

In [ ]:
display(pg.corr(dataI['petal_length'], dataI['petal_width'], method='pearson')) # Entre dos varibales, H0 -> Variables Independientes
display(sns.heatmap(dataI.corr(), vmin=-1, vmax=1, cmap='RdYlGn', annot=True)) # Todos contra todos

### 3) Bi-Variate Normality
Multivariate Normality in Variables

In [ ]:
# Test analitico para Multi
print(pg.multivariate_normality(dataI.drop(columns='species'))) # Henze-Zirkler Test, H0 -> Multiv Normal Dist


# Tests analiticos para Single
display(pg.normality(dataI['sepal_width'])) # Shapiro Test(single Var), H0 -> Normal Dist

# Otros tests
# sm.stats.diagnostic.normal_ad(dataI['sepal_width']) # Anderson-Darling StatsModels
# st.anderson(dataI['sepal_width'], dist='norm') # Anderson-Darling SciPy
# st.kstest(dataI['sepal_width'], 'norm')# Kolmogorov-Smirnov SciPy

# Test grafico para Single
pg.qqplot(dataI['sepal_width']) # QQ Plot
plt.show()

### 4) Residuals Normality With 0 Mean
Model residuals should follow a normal distribution, with 0 as mean

In [ ]:
# Primero hay que generar el Modelo de Regresion Lineal
display(pg.normality(model_colest.resid))
print('Mean:', model_colest.resid.mean())

# If distribution is not normal, regression results will be biased 
# and it may highlight that there are outliers or other assumptions being violated

### 5) Residuals Variance (Homosedasticity)
The variance of residual is the same for any value of x

In [ ]:
# Analsis grafico a mano usando Matplotlib
plt.scatter(x=model_colest.predict(), y=model_colest.resid)
plt.xlabel( 'Prediccion')
plt.ylabel('Residuo')
plt.title('Distribucion de Residuos')
plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
plt.show()
# No se observa estructura de embudo

# Analitico (Breusch-Pagan Test) -> H0: Residuos Homocedasticos
BP, p_value, _, _ =sm.stats.het_breuschpagan(model_colest.resid, model_colest.model.exog)
print('Estadistico BP y p-value:', BP, p_value)

# sns.residplot(data=dataP, x='edad', y='colest', lowess=True) # Pending
# Test White, Goldfeld-Quandt  Pending

### 6) Correlacion de Residuos
Aleatoriedad en el orden de las observacioens

In [ ]:
# Analsis grafico a mano usando Matplotlib
plt.scatter(x=dataP.index, y=model_colest.resid)
plt.xlabel( 'Index')
plt.ylabel('Residuo')
plt.title('Correlacion de Residuos')
plt.axhline(color='grey', linestyle='dashed', alpha=0.5)
plt.show()
# No se observa estructura

# Durbin-Watson Test: 2=No Correlacion (Independecia?)| 0=Correlacion Pos | 4=Correlacion Neg
print('Durbin-Watson:', sm.stats.durbin_watson(model_colest.resid)) # Sin validacion Estadistica

## 7) Transformacion Box-Cox (pendiente IC e IP)
Cuando los residuos no siguen una distribucion Normal, transformamos la y

In [ ]:
# Verificamos la Normalidad de los residuos
print(st.shapiro(model_cars.resid))
data = dataCars
y = data['dist']

# Box-Plot Transformation - Calculo Lambda
maxlog = st.boxcox(y)[1]
st.boxcox_normplot(y, -2, 2, plt)
plt.axvline(maxlog, color='red')
plt.show()

# Transfomracion: Si Lambda  = 0 -> np.log10(y) | y**Lambda
y_trans = pd.Series(y**.5, name='y_trans')
# No olvidarse de anti-transformar para los IC(10**predicted)

# Generamos el nuevo Dataset con la y transformada
data = pd.concat([data, y_trans], axis=1)

# Generamos el nuevo modelo con la y_transformada
model_cars_trans = smf.ols('y_trans ~ speed', data=data).fit()

# Verificamos la nueva normalidad de los residuos
print(pg.normality(model_cars_trans.resid))